# Metody Obliczeniowe w Nauce i Technice
## Laboratorium 4 - Singular Value Decomposition (Wyszukiwarka)
### Albert Gierlach

### 1. Przygotowanie danych
Dane przygotowano za pomocą wiki-crawlera (1200 artykułów z Wikipedii). Wykorzystano skrypt w Pythonie (https://github.com/bornabesic/wikipedia-crawler), dostosowując go do potrzeb zadania (dodanie opcji, która pozwala pobrać N artykułów). Źródła (wikipedia.py oraz crawler.py) są dostępne w archiwum z zadaniem.

Użycie:
```
python crawler.py N subdomain
```
gdzie N to liczba dokumentow do pobrania, a 'subdomain' to subdomena (użyto wartości 'en').
Dla polepszenia rezultatów zapewniono, że długość artykułu będzie większa niż 200 znaków.

Dane w formacie .txt pobierane są do folderu ./data

### 2., 3. Określenie bag-of-words 
Stworzono klasę, która będzie przechowywać dane jednego dokumentu oraz odpowiednie jej metody, które będą wykorzystane później. Odrzucono kilka słów, które powinny zostać zignorowane podczas wyszukiwania artykułów. Stworzono także klasę, która będzie odpowiadać za cache'owanie wyliczonych wektorów i macierzy, gdyż operacja ta trwa dość długo. Zastosowanie takiej klasy pozwala na jednokrotne wyliczenie wartości, a później wystarczy wczytać gotowe dane. Pierwsze uruchomienie trwa maksymalnie 5 minut. Wielkość cache około 500MB (jeśli zastosowanoby kompresję rozmiar zmniejszyłby się do 5MB, gdyż zawartość plików to głownie zera).

In [1]:
from collections import Counter
from typing import List, Any
from scipy import sparse
from scipy.sparse.linalg import svds
import os
import pickle
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import numpy as np
import operator

data_dir = "./data"
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rivit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rivit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class CacheManager:
    cache_dir = "./cache"  # place for storing calculated matrices, etc

    def __init__(self):
        self.loaded = set()

        if not os.path.exists(CacheManager.cache_dir):
            os.makedirs(CacheManager.cache_dir)

    def was_loaded(self, filename):
        return filename in self.loaded

    def save(self, filename, object):
        if self.was_loaded(filename):
            return

        try:
            with open('{}/{}'.format(CacheManager.cache_dir, filename), "wb") as f:
                pickle.dump(object, f, protocol=pickle.HIGHEST_PROTOCOL)
                print("> caching " + filename)
        except:
            return

    def load(self, filename):
        try:
            with open('{}/{}'.format(CacheManager.cache_dir, filename), "rb") as f:
                res = pickle.load(f)
                print("> using cached " + filename)
                self.loaded.add(filename)
                return res
        except:
            return None

class ArticleData:
    ignored_words = ["the", "for", "are", "you"]  # and probably more

    def __init__(self, title):
        self.title = title.split('.')[0]
        self.bag_of_words = Counter()
        self.words_vec = None
        self.words_vec_norm = None

    def load_bag_of_words(self, path):
        with open(path, "rt", encoding='utf-8') as f:
            lemmatizer = WordNetLemmatizer()
            words = re.findall(r'\w+', f.read().lower())
            loaded_words = [lemmatizer.lemmatize(word) for word in words if len(word) > 2 and word not in stop_words]
            self.bag_of_words.update(loaded_words)

        for ignore_token in ArticleData.ignored_words:
            del self.bag_of_words[ignore_token]

    def create_full_bag_of_words(self, keyset, size):
        self.words_vec = np.zeros(size)  # d_j
        for i, k in enumerate(keyset):
            self.words_vec[i] = self.bag_of_words[k]

        self.words_vec_norm = np.linalg.norm(self.words_vec)

    def print_contents(self):
        with open('{}/{}.txt'.format(data_dir, self.title), "rt", encoding='utf-8') as f:
            print(f.read())

    def normalize_word_vec(self):
        self.words_vec = self.words_vec / np.linalg.norm(self.words_vec)

In [3]:
cache = CacheManager()

articles_data: List[ArticleData] = cache.load('articles_data.dump')
if articles_data is None:
    articles_data = []
    for file in os.listdir(data_dir):
        a_data = ArticleData(file)
        a_data.load_bag_of_words("{}/{}".format(data_dir, file))
        articles_data.append(a_data)
print("total number of articles {}".format(len(articles_data)))

total_bag_of_words: Counter = cache.load('total_bag_of_words.dump')
if total_bag_of_words is None:
    total_bag_of_words = Counter()
    for article in articles_data:
        total_bag_of_words += article.bag_of_words

sizeof_total = len(total_bag_of_words)
wordset: List[Any] = cache.load('wordset.dump')
if wordset is None:
    wordset = list(total_bag_of_words.keys())
print("total number of words: {}".format(sizeof_total))

if not cache.was_loaded('articles_data.dump'):
    print("creating bag of words for every article")
    for article in articles_data:
        article.create_full_bag_of_words(wordset, sizeof_total)
print("created {} bags, every has {} elements".format(len(articles_data), sizeof_total))

total number of articles 1218
total number of words: 43446
creating bag of words for every article
created 1218 bags, every has 43446 elements


### 4., 5.  Rzadka macierz wektorów cech oraz IDF
Do budowy rzadkiej macierzy wykorzystano funckję crs_matrix(), która jest optymalizowana pod kątem przechowywania zer w wierszach.

In [4]:
def getIDF(wordset, articles_data):
    articles_num = len(articles_data)
    idf = []
    for word in wordset:
        cnt = 0
        for article in articles_data:
            if article.bag_of_words[word] != 0:
                cnt += 1

        idf.append(np.log10(articles_num/cnt))

    return idf


def create_sparse(articles_data, sizeof_total, idf):
    row = []
    column = []
    data = []

    for i in range(len(articles_data)):
        article = articles_data[i]
        for j in range(sizeof_total):
            if article.words_vec[j] != 0:
                row.append(j)
                column.append(i)
                data.append(article.words_vec[j] * idf[j])


    term_by_document_matirx = sparse.csr_matrix((data, (row, column)), shape=(sizeof_total, len(articles_data)))
    return term_by_document_matirx

In [5]:
idf: List[Any] = cache.load('idf.dump')
if idf is None:
    print('calculating idf')
    idf = getIDF(wordset, articles_data)

term_by_document_matirx: sparse.csr_matrix = cache.load('term_by_document_sparse_matrix.dump')
if term_by_document_matirx is None:
    print('creating sparse matrix')
    term_by_document_matirx = create_sparse(articles_data, sizeof_total, idf)
print("term by document matrix size: {}x{}".format(term_by_document_matirx.shape[0],
                                                   term_by_document_matirx.shape[1]))

calculating idf
creating sparse matrix
term by document matrix size: 43446x1218


In [6]:
cache.save('articles_data.dump', articles_data)
cache.save('wordset.dump', wordset)
cache.save('term_by_document_sparse_matrix.dump', term_by_document_matirx)
cache.save('total_bag_of_words.dump', total_bag_of_words)
cache.save('idf.dump', idf)

> caching articles_data.dump
> caching wordset.dump
> caching term_by_document_sparse_matrix.dump
> caching total_bag_of_words.dump
> caching idf.dump


### 6.  Program pozwalający na wyszukiwanie artykułów
Dla czytelności wyłączono wypisywanie całej treści artykułów

In [7]:
def parse_query(query, word_list):
    query = query.lower()
    words_dict = {word: index for index, word in enumerate(word_list)}
    words = re.findall(r'\w+', query)

    vec_query = np.zeros(len(word_list), dtype=int)
    for w in words:
        if w in words_dict.keys():
            vec_query[words_dict[w]] += 1

    if not np.any(vec_query):
        print("No results")
        return

    return vec_query


def print_search_results(res, k, query):
    res.sort(key=operator.itemgetter(0), reverse=True)
    print("Found articles for query [{}]:".format(query))
    for res_entry in res[:k]:
        print('> ' + res_entry[1].title.replace("_", " "))

#     print("\n\nFull articles:")
#     for res_entry in res[:k]:
#         print(res_entry[1].print_contents())
#         print('\n')
#         print('-' * 40)


def do_query(query, k, word_list, articles):
    vec_query = parse_query(query, word_list)

    q_norm = np.linalg.norm(vec_query)
    vec_query = vec_query.T
    res = []
    for a in articles:
        divider = q_norm * a.words_vec_norm
        prod = vec_query @ a.words_vec
        cos_theta = prod / divider
        res.append((cos_theta, a))

    print_search_results(res, k, query)
        
        
        
# reassign variables, just for readibility
# articles - list with all of documents (words vectors + bag of words)
# word_list - bag_of_words_dict.keys()
# A - sparse matrix, columns are words vectors from articles_data
articles, word_list, A = articles_data, wordset, term_by_document_matirx

### Przykładowe wyszukania

In [8]:
do_query("Action film", 5, word_list, articles)

Found articles for query [Action film]:
> The Blue Mansion
> Doni Sagali
> Ostend Film Festival
> Alankrita Shrivastava
> LA Rebellion


In [9]:
do_query("Winston Churchill", 5, word_list, articles)

Found articles for query [Winston Churchill]:
> Lancelot Royle
> No 47 Royal Marine Commando
> Johnny Murtagh
> 12th Yorkshire Parachute Battalion
> 1828 United States presidential election in Ohio


In [10]:
do_query("Beautiful places on earth", 5, word_list, articles)

Found articles for query [Beautiful places on earth]:
> Tolkiens legendarium
> Fossil Fighters
> Nanorchestidae
> 7 Persei
> Ek Khiladi Ek Haseena TV series


### Interaktywna wyszukiwarka

In [11]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Text, Label, IntSlider, Output
from IPython.display import display, clear_output

def btn(b):
    output.clear_output()
    with output:
        how_many = form.children[0].children[1].value
        text_to_search = form.children[1].children[1].value
        if len(text_to_search) > 1:
            do_query(text_to_search, how_many, word_list, articles)
        else:
            output.append_stdout("Text is too short")

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Results num'), IntSlider(min=1, max=30, value=10, descritpion='k_')], layout=form_item_layout),
    Box([Label(value='Query'), Text(placeholder="Wpisz zapytanie", descritpion='query_')], layout=form_item_layout),
    Box([Label(), Button(description="Search!")], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='50%'
))
output = Output()
form.children[2].children[1].on_click(btn)
form # you may need to run this cell manually

Box(children=(Box(children=(Label(value='Results num'), IntSlider(value=10, max=30, min=1)), layout=Layout(dis…

In [12]:
display(output) # place for results

Output()

### 7. Normalizacja wektorów
Znormalizowano wektory przechowywane w klasie ArticleData oraz zbudowano na nowo macierz rzadką A używając nowych wektorów. Wektor zapytania także został znormalizowany. Wykonano takie same wyszukiwania jak w wariancie bez normalizacji w celu weryfikacji poprawności.

In [13]:
def normalize_vectors(articles):
    for a in articles:
        a.normalize_word_vec()
        

normalize_vectors(articles)
A_normalized: sparse.csr_matrix = cache.load('A_normalized.dump')
if A_normalized is None:
    print('calculating new sparse matrix with new vectors')
    A_normalized = create_sparse(articles, len(word_list), idf)
    cache.save('A_normalized.dump', A_normalized)

calculating new sparse matrix with new vectors
> caching A_normalized.dump


In [14]:
def do_query2(query, k, word_list, articles, A):
    vec_query = parse_query(query, word_list)
    vec_query = vec_query / np.linalg.norm(vec_query)
    
    res = vec_query.T @ A
    probabilities = []
    for i, cos_theta in enumerate(res):
        probabilities.append((cos_theta, articles[i]))

    print_search_results(probabilities, k, query)

In [15]:
do_query2("Action film", 5, word_list, articles, A_normalized)

Found articles for query [Action film]:
> The Blue Mansion
> Doni Sagali
> Ostend Film Festival
> Alankrita Shrivastava
> LA Rebellion


In [16]:
do_query2("Winston Churchill", 5, word_list, articles, A_normalized)

Found articles for query [Winston Churchill]:
> Lancelot Royle
> No 47 Royal Marine Commando
> Johnny Murtagh
> 12th Yorkshire Parachute Battalion
> 1828 United States presidential election in Ohio


In [17]:
do_query2("Beautiful places on earth", 5, word_list, articles, A_normalized)

Found articles for query [Beautiful places on earth]:
> Tolkiens legendarium
> Fossil Fighters
> Nanorchestidae
> 7 Persei
> Ek Khiladi Ek Haseena TV series


### 8. Normalizacja wektorów
Zastosowanie SVD, low rank approximation oraz nowej miary prawdopodobieństwa.

In [18]:
def getSVD(A, rank):
    U, S, VT = sparse.linalg.svds(A, rank)
    return U @ np.diag(S) @ VT


def do_query3(query, k, word_list, articles, A, rank):
    vec_query = parse_query(query, word_list)
    q_norm = np.linalg.norm(vec_query)

    A_k = getSVD(A, rank)

    res = []
    for i, ak_row in enumerate(A_k.T):
        prod = vec_query.T @ ak_row
        cos_fi = prod / (q_norm * np.linalg.norm(ak_row))
        res.append((cos_fi, articles[i]))

    print_search_results(res, k, query)

In [19]:
rank = 120
do_query3("Action film", 5, word_list, articles, A, rank)

Found articles for query [Action film]:
> Ömer Faruk Sorak
> The Murderer Dimitri Karamazov
> Lakshmipati
> Aashiqui 2015 film
> Doni Sagali


In [20]:
do_query3("Winston Churchill", 5, word_list, articles, A, rank)

Found articles for query [Winston Churchill]:
> No 47 Royal Marine Commando
> Pseudoalteromonas denitrificans
> Johnny Murtagh
> List of Royal Air Force operations
> K1 World Grand Prix 2000 Final


In [21]:
do_query3("Beautiful places on earth", 5, word_list, articles, A, rank)

Found articles for query [Beautiful places on earth]:
> Tolkiens legendarium
> Fossil Fighters
> Elves Hill
> Crassispira martiae
> Zeder


### 9. Porównanie wyników z odszumianiem i bez. Wpływ IDF na wyniki.
Jak widać wyniki wyszukiwań w trzech przypadkach są bardzo zbliżone do siebie. Doświadczalno wybrano k=120 jako najmniejszą i nalepszą wartość, dla której wyniki są akceptowalne. Wyniki z odszumianiem są bardziej trafne niż bez odszumiania, jednak koszt obliczania SVD dla dużych macierzy może być odczuwalny. Przekształcenie IDF pozwala na redukcję znaczenia słów, które występują wielokrotnie w dokumentach, przez co wyniki są bardziej konkretne.

### Źródła:
* [Latent semantic indexing](https://nlp.stanford.edu/IR-book/html/htmledition/latent-semantic-indexing-1.html)
* [Latent Semantic Analysis](https://www.engr.uvic.ca/~seng474/svd.pdf)